In [1]:
'''
Cases we want to test: (Progress: 1/6)


# 1 instantaneity:  (DONE)
does setspeed change the velocity instantaneously?
when we setspeed, is the velocity of the direct next taken as the input velocity? What exactly happens?
Sample Test: ---- Car A ----- (NO OTHER CARS)
    Give Car A velociy, record step number at which the velocity was given, detect the velocity
    taken in each subsequent step and see when the velocity reaches our desired velocity.

# 2 setspeed: 
when given certain speed that is infeasible.. will sumo just disregard our commands OR
reach the closest speed it could to the required velocity ?
Sample Test: Car A ---- Car B ----
    velocity of B is 10 m/s. velocity of A starts at 3 m/s and is then set to 20 m/s.. does the
    car A reach velocity 10 ? or is the car going to maintain its old 3 m/s? 
    Hopefully it will reach 10 m/s. This is the logic we need in our ambulance.


# 3 safe distance:
how is the safe distance calculated? using MinGap ? Do we want to make it zero?
* Sample Test (Several Trials): ---- Car A ----- Car B --- 
    Car A moves at velocity = 10 m/s (test_amb_vel),  
        Car B is inserted at distance = 10 m (car_B_dist) infront of the ambulance, with velocity equal to 2 (car_B_vel)
    the trial is repeated for different Car A (Ambulance) velocities (10,8,6,4,2)
    the trial is repeated for different Car B start positions: test_amb_vel*1, test_amb_vel*1.5, test_amb_vel*2
    the trial is repeated for different Car B velocitiies: (0, 0.5*test_amb_vel, 2*test_amb_vel)
    
If any insertion approach is followed, check out:
https://sumo.dlr.de/docs/Simulation/VehicleInsertion.html

Must know how car follower model works. (ctrl+f carfollow)
Start Link: https://sumo.dlr.de/docs/Definition_of_Vehicles,_Vehicle_Types,_and_Routes.html


# 4 window: 
* how to control the size of the window seen by the ambulance ? I.e. what factors judge the minimum
distance after which the ambulance will start to change velocity 
*Read about MinGap and car follower model. (ctrl+f carfollow)
Start Link:https://sumo.dlr.de/docs/Definition_of_Vehicles,_Vehicle_Types,_and_Routes.html
*Sample Test (Several Tests): Car A ----- Car B
    Given Car A moves at 10 m/s (test_amb_vel), insert Car B at distance= 5 m (car_B_dist) with velocity = 5 m/sec (car_B_vel).
        If any delay occurs in insertion, this means that this distance is within the ambulance's window. Try for different
         * test_amb_vel: (10,5)
         * car_B_vel: (5,50)
         * car_B_dist: (2,5,10,15,20,30,40,50,60,70,80,90)
    

# 5 reward:  
* when to reward the agent..
now, since we do not apply acceleration to the ambulance, but rather wait for it to accelerate..
    we have a latency in our observation.. is this latency of one step? two steps? when do we notice
    the result of an acceleration action taken by a vehicle ?
* we must apply the reward when the action's corresponding observation is seen
* What we want to measure here is not the window size, but rather: given that we know the ambulance
    MUST repond (i.e. the agent is well within the window size to affect the ambulance).. when is the
    ambulance change of acceleration detected ?
* Sample Test: --- Car A --- Car B --- 
    Car A and Car B should first be close, after then, Car B wil stop suddenly and we will check when
    (after how many steps from the stop of Car B) can we detect the change in velocity of car A?
    
    Getting the  cars close: cars start at distance = x m, with both their velocities being x m/s


# 6 longitudinal feasbility:
How to check feasibility for increasing/decreasing speed? 
We want to make sure that our feasbility check is (if not like SUMO's) at least safer than SUMO's
check, to avoid giving the agent 
* Sample Test: To be determined after 1 to 5 are made.

'''

"\nCases we want to test:\n\n\n# 1 instantaneity: \ndoes setspeed change the velocity instantaneously?\nwhen we setspeed, is the velocity of the direct next taken as the input velocity? What exactly happens?\nSample Test: ---- Car A ----- (NO OTHER CARS)\n    Give Car A velociy, record step number at which the velocity was given, detect the velocity\n    taken in each subsequen step and see when the velocity reaches our desired velocity.\n\n# 2 setspeed: \nwhen given certain speed that is infeasible.. will sumo just disregard our commands OR\nreach the closest speed it could to the required velocity ?\nSample Test: Car A ---- Car B ----\n    velocity of B is 10 m/s. velocity of A starts at 3 m/s and is then set to 20 m/s.. does the\n    car A reach velocity 10 ? or is the car going to maintain its old 3 m/s? \n    Hopefully it will reach 10 m/s. This is the logic we need in our ambulance.\n\n\n# 3 safe distance:\nhow is the safe distance calculated? using MinGap ? Do we want to make it

# 1 instantaneity: 
does setspeed change the velocity instantaneously? <br>
when we setspeed, is the velocity of the direct next taken as the input velocity? What exactly happens?<br>
Sample Test: ---- Car A ----- (NO OTHER CARS)<br>
&emsp; Give Car A velociy, record step number at which the velocity was given, detect the velocity<br>
&emsp; taken in each subsequent step and see when the velocity reaches our desired velocity.<br>

In [3]:
'''
Set speed sets the speed to travel the upcoming step, instantaneously, on the condition that:
* If speed is negative, it is ignored, and maximum acceleration is applied by default. So:
    ** if velocity = 0.0, acc = -2, setspeed(-2): speed will be 0+max_acceleration over next step. 
        Which is wrong. Therefore max(0,speed+acc)
        should be applied.
        Expression was: min(int(self.spd  + acc), int(self.max_speed)) --> So as to not surpass max velocity
        Now it sould be:  max( 0.0 , min(int(self.spd  + acc), int(self.max_speed))) --> So as to not request negative velocity and 
                                                                                         have the request ignored.
                                                                                         
* No need to account for surpassing maximum velocity, SUMO already handles it.
* Maximum default negative acceleration is 4.5 ? Not sure
* Maximum default positive acceleration is 2.6 

*Conclusion: To have acceleration within range, set accel and decel within the XML files and format input to set Speed as follows:
    ** Input to set speed:  max( 0.0 , int(self.spd  + acc))
    ** XML: (Note the added accel and decel, decel = "1" means that maximum breaking is done at -1 m/s^2)
    <vType id="Ego" length="2.0" maxSpeed="10.0" sigma="0.0" color='0,0,1' accel="1" decel="1" />

'''

'\nSet speed sets the speed to travel the upcoming step, instantaneously, on the condition that:\n* If speed is negative, it is ignored, and maximum acceleration is applied by default. So:\n    ** if velocity = 0.0, acc = -2, setspeed(-2): speed will be 0+max_acceleration over next step. \n        Which is wrong. Therefore min(0,speed+acc)\n        should be applied.\n        Expression was: min(int(self.spd  + acc), int(self.max_speed)) --> So as to not surpass max velocity\n        Now it sould be:  max( 0.0 , min(int(self.spd  + acc), int(self.max_speed))) --> So as to not request negative velocity and \n                                                                                         have the request ignored.\n                                                                                         \n* No need to account for surpassing maximum velocity, SUMO already handles it.\n* Maximum default negative acceleration is 4.5 ? Not sure\n* Maximum default positive acceleration